In [77]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import get_openai_callback
import dotenv
dotenv.load_dotenv()

True

In [57]:
chat = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo"
)


In [59]:
from langchain.schema import BaseOutputParser
import re

class OutputParser(BaseOutputParser):
    def parse(self, text):
        reasoning = None
        answer = None

        # Reasoning 파싱
        reasoning_match = re.search(r'Reasoning:\s*(.*?)(?=Answer:)', text, re.DOTALL)
        if reasoning_match:
            reasoning = reasoning_match.group(1).strip()

        # Answer 파싱
        answer_match = re.search(r'Answer:\s*(.*)', text, re.DOTALL)
        if answer_match:
            answer = answer_match.group(1).strip()

        return {
            "reasoning": reasoning,
            "answer": answer
        }

In [60]:
system = "You are a medical expert. You are good at solving medical domain questions."
human = "{Instruction} \n {Question} \n {Context} \n {Options} \n {few_shots}\n" 
# human = "{text}"
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", human)
])

chain = prompt_template | chat | OutputParser()


### MedQA Dataset Load

In [6]:
import jsonlines

with jsonlines.open('./Dataset/MedQA/questions/US/test.jsonl') as reader:
    medqa_dataset = [obj for obj in reader]
medqa_dataset[0]

{'question': 'A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?',
 'answer': 'Tell the attending that he cannot fail to disclose this mistake',
 'options': {'A': 'Disclose the error to the patient but leave it out of the operative report',
  'B': 'Disclose the error to the patient and put it in the operative report',
  'C': 'Tell the attending that he cannot fail to disclose this mistake',
  'D': 'Report the physician to the ethics commi

### MedMCQA Dataset Load


In [88]:
import json

medmcqa_dataset = json.load(open('./Dataset/MedMCQA/train.json'))
medmcqa_dataset[0]

{'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'cop': 3,
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract',
 'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'choice_type': 'single'}

### PumMedQA Dataset Load


In [107]:
pubmedqa_dataset = json.load(open('./Dataset/PubMedQA/Labeled/ori_pqal_array.json'))

pubmedqa_dataset[0]

{'pmid': '21645374',
 'QUESTION': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?',
 'CONTEXTS': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.',
  'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early

In [47]:
def q_format_MedQA(item):
    ## context = [Question, Context, Options]
    context = []
    context.append(f"Question: {item["question"]}")
    context.append("")
    # option

    option_string = "Options:\n"
    option_string += f"A: {item['options']['A']}\n"
    option_string += f"B: {item['options']['B']}\n"
    option_string += f"C: {item['options']['C']}\n"
    option_string += f"D: {item['options']['D']}\n"
    context.append(option_string)
    return context
q_format_MedQA(medqa_dataset[0])

['Question: A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?',
 '',
 'Options:\nA: Disclose the error to the patient but leave it out of the operative report\nB: Disclose the error to the patient and put it in the operative report\nC: Tell the attending that he cannot fail to disclose this mistake\nD: Report the physician to the ethics committee\n']

In [48]:
def q_format_MedMCQA(item):
    context = []
    context.append(f"Question: {item["question"]}")
    context.append("")

    #option
    option_string = "Options:\n"
    option_string += f"A: {item['opa']}\n"
    option_string += f"B: {item['opb']}\n"
    option_string += f"C: {item['opc']}\n"
    option_string += f"D: {item['opd']}\n"
    context.append(option_string)
    return context
q_format_MedMCQA(medmcqa_dataset[0])

['Question: Which of the following is derived from fibroblast cells ?',
 '',
 'Options:\nA: TGF-13\nB: MMP2\nC: Collagen\nD: Angiopoietin\n']

In [49]:
def q_format_PubMedQA(item):
    context = []
    context.append(f"Question: {item["QUESTION"]}")
    context.append(f"Context: {item["CONTEXTS"][0]}")
    #option
    option_string = "Options:\n"
    option_string += "A: yes\n"
    option_string += "B: no\n"
    option_string += "C: maybe\n"
    context.append(option_string)
    return context
q_format_PubMedQA(pubmedqa_dataset[0])    

['Question: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?',
 'Context: Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.',
 'Options:\nA: yes\nB: no\nC: maybe\n']

In [109]:
# Complete Prompt

instruction = "Instruction: Solve the following medical question and generate your reasoning step by step and answer with the following format, 'Reasoning: your reasoning step \nAnswer: correct_option(correct_option_value). Do NOT make additional response."
question, context, options = q_format_PubMedQA(pubmedqa_dataset[10])
formatted_prompt = prompt_template.format_messages(Instruction=instruction, Question=question, Context= context, Options=options,few_shots="")
reasoning_answer = chain.invoke({"Instruction": instruction, "Question" : question, "Context": context, "Options": options, "few_shots":""})
reasoning_answer["answer"]

'A: yes.'

### Parsing

In [106]:
from tqdm import tqdm
results_collected_medqa = []
pass_collected_medqa = []
with get_openai_callback() as usage:
    for i in tqdm(range(200)):
        cur_question, cur_context, cur_options = q_format_MedQA(medqa_dataset[i])
        cur_correct_answer = medqa_dataset[i]["answer_idx"]
        # Chat
        reasoning_answer = chain.invoke({"Instruction": instruction, "Question" : cur_question, "Context": cur_context, "Options": cur_options, "few_shots":""})
        correctness = ((cur_correct_answer + ":") in reasoning_answer["answer"].split()[0]) or (medqa_dataset[i]["answer_idx"] in reasoning_answer["answer"])
        pass_collected_medqa.append(correctness)
        if (i == 199):
            results_collected_medqa.append(medqa_dataset[i] | {"reasoning" : reasoning_answer["reasoning"], "model chosen" : reasoning_answer["answer"], "correctness" : correctness, "accuracy": sum(pass_collected_medqa)/ len(pass_collected_medqa)})
        else:
            results_collected_medqa.append(medqa_dataset[i] | {"reasoning" : reasoning_answer["reasoning"], "model chosen" : reasoning_answer["answer"], "correctness" : correctness})

        if (i + 1) % 10 == 0:
            with open(f'results_collected_medqa.json', 'a') as f:
                json.dump(results_collected_medqa, f, ensure_ascii=False, indent=4)
            results_collected_medqa = []
        print(f"Acc: {sum(pass_collected_medqa)/ len(pass_collected_medqa)}")
    print(usage)

        

  0%|          | 1/200 [00:01<05:51,  1.77s/it]

Acc: 0.0


  1%|          | 2/200 [00:04<07:52,  2.38s/it]

Acc: 0.0


  2%|▏         | 3/200 [00:09<11:02,  3.36s/it]

...

Acc: 0.6464646464646465


100%|█████████▉| 199/200 [09:01<00:02,  2.73s/it]

Acc: 0.6432160804020101


100%|██████████| 200/200 [09:03<00:00,  2.72s/it]

Acc: 0.645
Tokens Used: 86484
	Prompt Tokens: 55464
	Completion Tokens: 31020
Successful Requests: 200
Total Cost (USD): $0.07426199999999997


In [103]:
pass_collected_medmcqa = []
results_collected_medmcqa = []
value_to_letter = {
    1: 'A',
    2: 'B',
    3: 'C',
    4: 'D'
}
with get_openai_callback() as usage:
    for i in tqdm(range(200)):
        cur_question, cur_context, cur_options = q_format_MedMCQA(medmcqa_dataset[i])
        cur_correct_answer = value_to_letter[medmcqa_dataset[i]["cop"]]
        # Chat
        reasoning_answer = chain.invoke({"Instruction": instruction, "Question" : cur_question, "Context": cur_context, "Options": cur_options, "few_shots":""})
        correctness = ((cur_correct_answer + ":") in reasoning_answer["answer"].split()[0]) or (medmcqa_dataset[i]["op" + cur_correct_answer.lower()] in reasoning_answer["answer"])
        pass_collected_medmcqa.append(correctness)
        if (i == 199):
            results_collected_medmcqa.append(medmcqa_dataset[i] | {"reasoning" : reasoning_answer["reasoning"], "model chosen" : reasoning_answer["answer"], "correctness" : correctness, "accuracy": sum(pass_collected_medmcqa)/ len(pass_collected_medmcqa)})
        else:
            results_collected_medmcqa.append(medmcqa_dataset[i] | {"reasoning" : reasoning_answer["reasoning"], "model chosen" : reasoning_answer["answer"], "correctness" : correctness})

        if (i + 1) % 10 == 0:
            with open(f'results_collected_medmcqa.json', 'a') as f:
                json.dump(results_collected_medmcqa, f, ensure_ascii=False, indent=4)
            results_collected_medmcqa = []
        print(f"Acc: {sum(pass_collected_medmcqa)/ len(pass_collected_medmcqa)}")
    print(usage)

        

  0%|          | 1/200 [00:01<05:28,  1.65s/it]

Acc: 1.0


  1%|          | 2/200 [00:02<04:29,  1.36s/it]

Acc: 1.0


  2%|▏         | 3/200 [00:04<05:31,  1.68s/it]

...

Acc: 0.6464646464646465


100%|█████████▉| 199/200 [06:43<00:02,  2.16s/it]

Acc: 0.6482412060301508


100%|██████████| 200/200 [06:45<00:00,  2.03s/it]

Acc: 0.645
Tokens Used: 44862
	Prompt Tokens: 24888
	Completion Tokens: 19974
Successful Requests: 200
Total Cost (USD): $0.042405000000000005


### PumMedQA


In [112]:
value_to_letter_pm = {
    "yes": 'A',
    "no": 'B',
    "maybe": 'C'
}
results_collected_pubmedqa = []
pass_collected_pubmedqa = []
with get_openai_callback() as usage:
    for i in tqdm(range(200)):
        cur_question, cur_context, cur_options = q_format_PubMedQA(pubmedqa_dataset[i])
        cur_correct_answer = value_to_letter_pm[pubmedqa_dataset[i]["final_decision"]]

        # Chat
        reasoning_answer = chain.invoke({"Instruction": instruction, "Question" : cur_question, "Context": cur_context, "Options": cur_options, "few_shots":""})
        correctness = ((cur_correct_answer + ":") in reasoning_answer["answer"].split()[0]) or (pubmedqa_dataset[i]["final_decision"] in reasoning_answer["answer"])
        pass_collected_pubmedqa.append(correctness)
        if (i == 199):
            results_collected_pubmedqa.append(pubmedqa_dataset[i] | {"reasoning" : reasoning_answer["reasoning"], "model chosen" : reasoning_answer["answer"], "correctness" : correctness, "accuracy": sum(pass_collected_pubmedqa)/ len(pass_collected_pubmedqa)})
        else:
            results_collected_pubmedqa.append(pubmedqa_dataset[i] | {"reasoning" : reasoning_answer["reasoning"], "model chosen" : reasoning_answer["answer"], "correctness" : correctness})

        if (i + 1) % 10 == 0:
            with open(f'results_collected_pubmedqa.json', 'a') as f:
                json.dump(results_collected_pubmedqa, f, ensure_ascii=False, indent=4)
            results_collected_pubmedqa = []
        print(f"Acc: {sum(pass_collected_pubmedqa)/ len(pass_collected_pubmedqa)}")
    print(usage)

        

  0%|          | 1/200 [00:01<05:47,  1.75s/it]

Acc: 1.0


  1%|          | 2/200 [00:04<07:44,  2.35s/it]

Acc: 0.5


  2%|▏         | 3/200 [00:06<06:38,  2.02s/it]

...

Acc: 0.4797979797979798


100%|█████████▉| 199/200 [06:45<00:01,  1.75s/it]

Acc: 0.47738693467336685


100%|██████████| 200/200 [06:48<00:00,  2.04s/it]

Acc: 0.48
Tokens Used: 56657
	Prompt Tokens: 34341
	Completion Tokens: 22316
Successful Requests: 200
Total Cost (USD): $0.05064450000000001
